In [1]:
import random
from typing import Dict, List, Union

from datasets import load_dataset
from fuzzywuzzy import fuzz, process
from sklearn.metrics import classification_report
from tqdm import tqdm
from transformers import pipeline

In [2]:
def prepare_message_for_llm(text: Union[str, List[str]], categories: Dict[str, str]) -> Dict[str, List[Dict[str, str]]]:
    if len(categories) < 2:
        raise RuntimeError(f'The category list is too small! Expected 2 or more categories, got {len(categories)} ones.')
    categories_ = sorted(list(categories.keys()))
    categories_as_string = ', '.join(categories_[:-1]) + ' и ' + categories_[-1]
    
    def build_prompt(txt: str) -> str:
        prompt = f'Определи тему текста из списка: {categories_as_string}. Ответь только названием темы.\n'
        for cat in categories_:
            prompt += f'Текст: {categories[cat]}\nТема: {cat}\n'
        prompt += f'Текст: {txt}\nТема:'
        return prompt

    if isinstance(text, str):
        messages = [
            {'role': 'system', 'content': 'Ты — эксперт по классификации текстов на русском языке.'},
            {'role': 'user', 'content': build_prompt(text)}
        ]
    else:
        messages = []
        for it in text:
            messages.append([
                {'role': 'system', 'content': 'Ты — эксперт по классификации текстов на русском языке.'},
                {'role': 'user', 'content': build_prompt(it)}  
            ])
    return {'message_for_llm': messages}

Модель возможно плохо воспринимает русский - попробуем английский

In [ ]:
def prepare_message_for_llm(text: Union[str, List[str]], category_examples: Dict[str, str]) -> Dict[str, List[Dict[str, str]]]:
    """Готовим промпт для LLM: строгий JSON-ответ + русские примеры по категориям.

    category_examples: словарь {имя_категории: пример_текста_этой_категории}
    """
    if len(category_examples) < 2:
        raise RuntimeError(
            f'The category list is too small! Expected 2 or more categories, got {len(category_examples)} ones.'
        )

    categories = sorted(list(category_examples.keys()))
    categories_str = ", ".join(f'"{cat}"' for cat in categories)

    def build_prompt(txt: str) -> str:
        prompt = (
            "Ты — эксперт по классификации новостных текстов на русском языке.\n"
            f"Твоя задача — отнести текст ровно к ОДНОЙ категории из списка: [{categories_str}].\n"
            'Ответь СТРОГО в формате валидного JSON: {"category": "<одна_категория_из_списка>"}.\n'
            "Не добавляй никакого другого текста, пояснений или комментариев.\n\n"
            "Примеры:\n"
        )

        # Few-shot
        for cat in categories:
            ex = category_examples[cat].replace("\n", " ")
            prompt += f'Текст: {ex}\nКлассификация: {{"category": "{cat}"}}\n\n'

        prompt += (
            "Теперь классифицируй следующий текст.\n"
            f"Текст: {txt}\n"
            "Классификация:"
        )
        return prompt

    if isinstance(text, str):
        messages = [
            {
                'role': 'system',
                'content': (
                    'Ты — эксперт по классификации текстов на русском языке. '
                    'Всегда отвечай строго в формате JSON вида {"category": "..."}.'
                ),
            },
            {'role': 'user', 'content': build_prompt(text)},
        ]
    else:
        messages = []
        for it in text:
            messages.append([
                {
                    'role': 'system',
                    'content': (
                        'Ты — эксперт по классификации текстов на русском языке. '
                        'Всегда отвечай строго в формате JSON вида {"category": "..."}.'
                    ),
                },
                {'role': 'user', 'content': build_prompt(it)},
            ])

    return {'message_for_llm': messages}

In [ ]:
from collections import Counter


def build_messages_with_prompt_version(text: str,
                                       category_examples: Dict[str, str],
                                       version: int = 1) -> List[Dict[str, str]]:
    categories = sorted(list(category_examples.keys()))
    categories_str = ", ".join(f'"{cat}"' for cat in categories)

    system_msg = {
        'role': 'system',
        'content': (
            'Ты — эксперт по классификации новостных текстов на русском языке. '
            'Всегда отвечай строго в формате JSON вида {"category": "..."}.'
        ),
    }

    if version == 1:
        prompt = (
            "Ты — эксперт по классификации новостных текстов на русском языке.\n"
            f"Твоя задача — отнести текст ровно к ОДНОЙ категории из списка: [{categories_str}].\n"
            'Ответь СТРОГО в формате валидного JSON: {"category": "<одна_категория_из_списка>"}.\n'
            "Не добавляй никакого другого текста, пояснений или комментариев.\n\n"
            "Примеры:\n"
        )
    elif version == 2:
        prompt = (
            "Тебе даны новости на русском языке. Твоя задача — определить, к какой теме относится каждая новость.\n"
            f"Выбери РОВНО одну тему из списка: [{categories_str}].\n"
            "Не придумывай свои темы, используй только темы из списка.\n"
            'Ответ верни только в виде JSON: {"category": "<одна_категория_из_списка>"}.\n'
            "Не добавляй вступлений, пояснений и другого текста.\n\n"
            "Ниже несколько примеров правильной разметки:\n"
        )
    else:  
        prompt = (
            f"Определи тему новости. Доступные темы: [{categories_str}].\n"
            'Ответь строго JSON: {"category": "<одна_категория_из_списка>"}.\n'
            "Если сомневаешься, выбери самую подходящую тему.\n\n"
            "Примеры:\n"
        )

    for cat in categories:
        ex = category_examples[cat].replace("\n", " ")
        prompt += f'Текст: {ex}\nКлассификация: {{"category": "{cat}"}}\n\n'

    prompt += (
        "Теперь классифицируй следующий текст.\n"
        f"Текст: {text}\n"
        "Классификация:"
    )

    user_msg = {'role': 'user', 'content': prompt}
    return [system_msg, user_msg]


# Pipeline
## Загрузим модель

In [5]:
llm_pipeline = pipeline(
    model='Qwen/Qwen1.5-1.8B-Chat',
    device_map='auto',
    trust_remote_code=True,
    return_full_text=False,
    max_new_tokens=30 
)

Device set to use cpu


 ## Загрузим датасет

In [6]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

## Выделим категории

In [7]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
print(f'Categories for classification are: {list_of_categories}')

Categories for classification are: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [8]:
# Маппинг: ключевые слова → официальная категория
category_aliases = {
    'entertainment': ['развлечения', 'кино', 'музыка', 'шоу', 'театр', 'игра', 'игры', 'entertain', 'show', 'music', 'film', 'movie'],
    'geography': ['география', 'страна', 'карта', 'регион', 'местность', 'гео', 'world', 'map', 'country'],
    'health': ['медицина', 'здоровье', 'болезнь', 'лечение', 'доктор', 'врач', 'здоров', 'health', 'medical', 'doctor', 'treatment'],
    'politics': ['политика', 'выборы', 'парламент', 'правительство', 'закон', 'партия', 'политик', 'politic', 'election', 'law', 'government'],
    'science/technology': ['наука', 'технологии', 'техника', 'tech', 'science', 'computer', 'it', 'инженерия', 'робот', 'роботы', 'технология'],
    'sports': ['спорт', 'футбол', 'баскетбол', 'матч', 'олимпиада', 'чемпионат', 'команда', 'игрок', 'sport', 'game', 'match', 'basketball', 'football'],
    'travel': ['путешествия', 'туризм', 'поездка', 'отпуск', 'гостиница', 'тур', 'trip', 'travel', 'vacation', 'hotel', 'journey']
}

In [9]:
print(validation_set)

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 99
})


## Выделим случайные примеры

In [10]:
examples_by_categories = dict()
for current_category in list_of_categories:
    examples_by_categories[current_category] = random.choice(
        train_set.filter(lambda it: it['category'] == current_category)['text']
    )
    print(f'Category: {current_category}')
    print(f'Random text: {examples_by_categories[current_category]}\n')

Category: entertainment
Random text: Джон Грант с канала WNED Buffalo (где показывают Reading Rainbow) сказал: "Шоу Reading Rainbow показало детям, зачем нужно читать, ... привило любовь к чтению — [оно] побудило детей взять книгу в руки и прочитать ее".

Category: geography
Random text: В разных местах на поверхности Луны учеными были выявлены формы рельефа, называемые лопастными уступами, которые скорей всего возникли в процессе очень медленного сокращения Луны.

Category: health
Random text: Состояние президента стабильное, но он самоизолируется на несколько дней у себя дома.

Category: politics
Random text: Представитель Буша Гордон Джондро назвал обещание Северной Кореи "большим шагом на пути к цели по достижению поддающегося проверке ядерного разоружения Корейского полуострова".

Category: science/technology
Random text: Серия ASUS Eee PC, ранее представленная во всём мире с ориентацией на такие факторы, как экономия средств и функциональность, стала горячей темой во время месяца

## Обернем тексты в prompt для llm

In [11]:
validation_set_for_llm = validation_set.map(lambda it: prepare_message_for_llm(it['text'], examples_by_categories))
test_set_for_llm = test_set.map(lambda it: prepare_message_for_llm(it['text'], examples_by_categories))

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [12]:
print(validation_set_for_llm)

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 99
})


In [13]:
print(validation_set['text'][0])

Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.


In [14]:
print(validation_set_for_llm['message_for_llm'][0])

[{'content': 'Ты — эксперт по классификации текстов на русском языке. Всегда отвечай строго в формате JSON вида {"category": "..."}.', 'role': 'system'}, {'content': 'Ты — эксперт по классификации новостных текстов на русском языке.\nТвоя задача — отнести текст ровно к ОДНОЙ категории из списка: ["entertainment", "geography", "health", "politics", "science/technology", "sports", "travel"].\nОтветь СТРОГО в формате валидного JSON: {"category": "<одна_категория_из_списка>"}.\nНе добавляй никакого другого текста, пояснений или комментариев.\n\nПримеры:\nТекст: Джон Грант с канала WNED Buffalo (где показывают Reading Rainbow) сказал: "Шоу Reading Rainbow показало детям, зачем нужно читать, ... привило любовь к чтению — [оно] побудило детей взять книгу в руки и прочитать ее".\nКлассификация: {"category": "entertainment"}\n\nТекст: В разных местах на поверхности Луны учеными были выявлены формы рельефа, называемые лопастными уступами, которые скорей всего возникли в процессе очень медленного

## Сгенерируем ответы

In [15]:
y_pred_raw = []
for msg in tqdm(validation_set_for_llm['message_for_llm']):
    try:
        output = llm_pipeline(msg, max_new_tokens=10)
        response = output[0]['generated_text'].strip()
    except Exception as e:
        print(f"Error: {e}")
        response = ""
    y_pred_raw.append(response)

y_true = validation_set['category']

100%|██████████| 99/99 [04:03<00:00,  2.46s/it]


Перепишем под новый prepare_message

In [ ]:
import json
import re

def parse_llm_output(raw_output: str, valid_categories: List[str]) -> str:
    """Надёжный парсинг: сначала JSON, потом regex, потом fuzzy."""
    raw_output = raw_output.strip()
    
    try:
        data = json.loads(raw_output)
        if isinstance(data, dict) and 'category' in data:
            cat = data['category'].strip()
            if cat in valid_categories:
                return cat
    except (json.JSONDecodeError, TypeError):
        pass

    match = re.search(r'"category"\s*:\s*"([^"]+)"', raw_output)
    if match:
        cat = match.group(1).strip()
        if cat in valid_categories:
            return cat

    best_match, score = process.extractOne(raw_output, valid_categories, scorer=fuzz.token_sort_ratio)
    return best_match if score > 30 else valid_categories[0]

y_pred_raw = []
for msg in tqdm(validation_set_for_llm['message_for_llm']):
    try:
        output = llm_pipeline(
            msg,
            max_new_tokens=20,
            do_sample=True,
            temperature=0.2,
            top_p=0.7,
        )
        response = output[0]['generated_text'].strip()
    except Exception as e:
        print(f"Error: {e}")
        response = ""
    y_pred_raw.append(response)

y_true = validation_set['category']
y_pred_clean = [parse_llm_output(pred, list_of_categories) for pred in y_pred_raw]
print(classification_report(y_true=y_true, y_pred=y_pred_clean))

100%|██████████| 99/99 [03:57<00:00,  2.40s/it]

                    precision    recall  f1-score   support

     entertainment       0.24      0.44      0.31         9
         geography       0.21      0.88      0.34         8
            health       0.83      0.45      0.59        11
          politics       1.00      0.36      0.53        14
science/technology       0.58      0.28      0.38        25
            sports       0.89      0.67      0.76        12
            travel       0.47      0.40      0.43        20

          accuracy                           0.44        99
         macro avg       0.60      0.50      0.48        99
      weighted avg       0.62      0.44      0.47        99



In [17]:
print("Пример ответа модели:", y_pred_raw[0])

Пример ответа модели: {"category": "fitness"}


 ## Постобработка текста

In [ ]:
import re

def map_to_category(raw_pred: str, aliases: dict, valid_categories: list) -> str:
    """Извлекает категорию через синонимы, затем fallback на fuzzy."""
    if not raw_pred.strip():
        return valid_categories[0]
    
    raw_low = raw_pred.strip().lower()
    if raw_low in [cat.lower() for cat in valid_categories]:
        for cat in valid_categories:
            if cat.lower() == raw_low:
                return cat

    for category, keywords in aliases.items():
        if any(kw in raw_low for kw in keywords):
            return category

    best_match, score = process.extractOne(raw_pred, valid_categories, scorer=fuzz.token_sort_ratio)
    return best_match if score > 30 else valid_categories[0]

y_pred_clean = [map_to_category(pred, category_aliases, list_of_categories) for pred in y_pred_raw]

In [19]:
from sklearn.metrics import confusion_matrix
import numpy as np

cm = confusion_matrix(y_true, y_pred_clean, labels=list_of_categories)
print("Confusion matrix (rows=real, cols=pred):")
print(cm)

Confusion matrix (rows=real, cols=pred):
[[4 1 0 0 0 1 3]
 [0 7 0 0 0 1 0]
 [2 0 5 0 3 0 1]
 [0 9 0 5 0 0 0]
 [4 9 1 0 7 1 3]
 [1 0 0 0 1 8 2]
 [1 6 0 0 2 3 8]]


In [ ]:
PROMPT_VERSIONS = [1, 2, 3]


def smart_majority_vote(labels: List[str], text: str = "") -> str:
    processed_labels = []
    for label in labels:
        processed = map_to_category(label, category_aliases, list_of_categories)
        processed_labels.append(processed)
    
    counts = Counter(processed_labels)
    most_common = counts.most_common()
    
    if len(most_common) > 0 and most_common[0][1] >= 2:
        return most_common[0][0]
    
    if text:
        text_low = text.lower()
        best_score = 0
        best_label = most_common[0][0] if most_common else list_of_categories[0]
        
        for label in processed_labels:
            if label in category_aliases:
                score = sum(1 for kw in category_aliases[label] if kw in text_low)
                if score > best_score:
                    best_score = score
                    best_label = label
        
        if best_score > 0:
            return best_label
    
    return most_common[0][0] if most_common else list_of_categories[0]


print("Проверяем качество каждого промпта отдельно на валидации\n")
prompt_scores = {}
for v in PROMPT_VERSIONS:
    val_preds_v = []
    for text in tqdm(validation_set['text'], desc=f"Prompt v{v}"):
        msgs = build_messages_with_prompt_version(text, examples_by_categories, version=v)
        try:
            output = llm_pipeline(
                msgs,
                max_new_tokens=20,
                do_sample=True,
                temperature=0.1,
                top_p=0.6,
            )
            raw = output[0]['generated_text'].strip()
        except Exception as e:
            raw = ""
        label = parse_llm_output(raw, list_of_categories)
        label = map_to_category(label, category_aliases, list_of_categories)
        val_preds_v.append(label)
    
    from sklearn.metrics import f1_score
    f1_macro = f1_score(validation_set['category'], val_preds_v, average='macro')
    prompt_scores[v] = f1_macro
    print(f"Prompt v{v}: macro F1 = {f1_macro:.3f}")

best_prompt = max(prompt_scores.items(), key=lambda x: x[1])[0]
print(f"\nЛучший промпт: v{best_prompt} (F1={prompt_scores[best_prompt]:.3f})\n")

print("Запускаем улучшенный ансамбль\n")
ensemble_val_preds = []
for text in tqdm(validation_set['text'], desc="Ensemble (val)"):
    per_prompt_labels = []
    for v in PROMPT_VERSIONS:
        msgs = build_messages_with_prompt_version(text, examples_by_categories, version=v)
        try:
            output = llm_pipeline(
                msgs,
                max_new_tokens=20,
                do_sample=True,
                temperature=0.1,
                top_p=0.6,
            )
            raw = output[0]['generated_text'].strip()
        except Exception as e:
            raw = ""
        label = parse_llm_output(raw, list_of_categories)
        per_prompt_labels.append(label)

    final_label = smart_majority_vote(per_prompt_labels, text)
    ensemble_val_preds.append(final_label)

print("\nУлучшенный Ensemble - метрики на валидации:\n")
print(classification_report(y_true=validation_set['category'], y_pred=ensemble_val_preds))

# Тест
ensemble_test_preds = []
for text in tqdm(test_set['text'], desc="Ensemble (test)"):
    per_prompt_labels = []
    for v in PROMPT_VERSIONS:
        msgs = build_messages_with_prompt_version(text, examples_by_categories, version=v)
        try:
            output = llm_pipeline(
                msgs,
                max_new_tokens=20,
                do_sample=True,
                temperature=0.1,
                top_p=0.6,
            )
            raw = output[0]['generated_text'].strip()
        except Exception as e:
            raw = ""
        label = parse_llm_output(raw, list_of_categories)
        per_prompt_labels.append(label)

    final_label = smart_majority_vote(per_prompt_labels, text)
    ensemble_test_preds.append(final_label)

print("\nУлучшенный Ensemble - метрики на тесте:\n")
print(classification_report(y_true=test_set['category'], y_pred=ensemble_test_preds))


Проверяем качество каждого промпта отдельно на валидации...



Prompt v1: 100%|██████████| 99/99 [03:34<00:00,  2.17s/it]


Prompt v1: macro F1 = 0.484


Prompt v2: 100%|██████████| 99/99 [03:43<00:00,  2.26s/it]


Prompt v2: macro F1 = 0.540


...

Prompt v3: macro F1 = 0.535

Лучший промпт: v2 (F1=0.540)

Запускаем улучшенный ансамбль...



Ensemble (val): 100%|██████████| 99/99 [10:56<00:00,  6.63s/it]



Улучшенный Ensemble — метрики на валидации:

                    precision    recall  f1-score   support

     entertainment       0.28      0.56      0.37         9
         geography       0.26      0.88      0.40         8
            health       0.83      0.45      0.59        11
          politics       1.00      0.43      0.60        14
science/technology       0.69      0.44      0.54        25
            sports       0.89      0.67      0.76        12
            travel       0.53      0.45      0.49        20

          accuracy                           0.52        99
         macro avg       0.64      0.55      0.53        99
      weighted avg       0.67      0.52      0.54        99



Ensemble (test): 100%|██████████| 204/204 [21:56<00:00,  6.46s/it]


Улучшенный Ensemble — метрики на тесте:

                    precision    recall  f1-score   support

     entertainment       0.26      0.47      0.33        19
         geography       0.33      0.88      0.48        17
            health       0.87      0.59      0.70        22
          politics       1.00      0.50      0.67        30
science/technology       0.89      0.49      0.63        51
            sports       0.90      0.72      0.80        25
            travel       0.69      0.78      0.73        40

          accuracy                           0.62       204
         macro avg       0.70      0.63      0.62       204
      weighted avg       0.76      0.62      0.64       204



In [ ]:
def apply_confusion_correction(pred: str, text: str) -> str:
    text_low = text.lower()
    if pred == 'health' and any(kw in text_low for kw in ['технолог', 'робот', 'компьютер', 'ai', 'искусствен', 'программ', 'код']):
        return 'science/technology'
    return pred

y_pred_corrected = [
    apply_confusion_correction(pred, text)
    for pred, text in zip(y_pred_clean, validation_set['text'])
]

y_pred_clean = y_pred_corrected

In [21]:
print(classification_report(y_true=y_true, y_pred=y_pred_clean))

                    precision    recall  f1-score   support

     entertainment       0.33      0.44      0.38         9
         geography       0.22      0.88      0.35         8
            health       0.83      0.45      0.59        11
          politics       1.00      0.36      0.53        14
science/technology       0.54      0.28      0.37        25
            sports       0.57      0.67      0.62        12
            travel       0.47      0.40      0.43        20

          accuracy                           0.44        99
         macro avg       0.57      0.50      0.47        99
      weighted avg       0.58      0.44      0.46        99



## Тестовые данные

In [22]:
y_test_pred_raw = []
for msg in tqdm(test_set_for_llm['message_for_llm']):
    try:
        output = llm_pipeline(msg, max_new_tokens=10)
        response = output[0]['generated_text'].strip()
    except Exception as e:
        print(f"Error: {e}")
        response = ""
    y_test_pred_raw.append(response)

y_test_true = test_set['category']

100%|██████████| 204/204 [08:30<00:00,  2.50s/it]


Для теста аналогичные изменения

In [23]:
y_test_pred_raw = []
for msg in tqdm(test_set_for_llm['message_for_llm']):
    try:
        output = llm_pipeline(
            msg,
            max_new_tokens=20,
            do_sample=True,
            temperature=0.2,
            top_p=0.7,
        )
        response = output[0]['generated_text'].strip()
    except Exception as e:
        print(f"Error: {e}")
        response = ""
    y_test_pred_raw.append(response)

y_test_true = test_set['category']
y_test_pred_clean = [parse_llm_output(pred, list_of_categories) for pred in y_test_pred_raw]
print(classification_report(y_true=y_test_true, y_pred=y_test_pred_clean))

100%|██████████| 204/204 [08:21<00:00,  2.46s/it]

                    precision    recall  f1-score   support

     entertainment       0.26      0.47      0.33        19
         geography       0.29      0.82      0.42        17
            health       0.87      0.59      0.70        22
          politics       1.00      0.43      0.60        30
science/technology       0.88      0.43      0.58        51
            sports       0.90      0.72      0.80        25
            travel       0.64      0.75      0.69        40

          accuracy                           0.58       204
         macro avg       0.69      0.60      0.59       204
      weighted avg       0.74      0.58      0.61       204



In [24]:
y_test_pred_clean = [
    process.extractOne(pred, list_of_categories, scorer=fuzz.token_sort_ratio)[0]
    if pred.strip() != ""
    else list_of_categories[0]
    for pred in y_test_pred_raw
]


In [25]:
print(classification_report(y_true=y_test_true, y_pred=y_test_pred_clean))

                    precision    recall  f1-score   support

     entertainment       0.26      0.47      0.33        19
         geography       0.29      0.82      0.42        17
            health       0.87      0.59      0.70        22
          politics       1.00      0.43      0.60        30
science/technology       0.88      0.43      0.58        51
            sports       0.90      0.72      0.80        25
            travel       0.64      0.75      0.69        40

          accuracy                           0.58       204
         macro avg       0.69      0.60      0.59       204
      weighted avg       0.74      0.58      0.61       204



In [ ]:
def normalize_and_map(pred: str) -> str:
    pred_clean = pred.strip().lower()
    
    if pred in list_of_categories:
        return pred

    norm_to_orig = {cat.lower().replace(" ", "").replace("/", ""): cat for cat in list_of_categories}
    norm_pred = pred_clean.replace(" ", "").replace("/", "")
    if norm_pred in norm_to_orig:
        return norm_to_orig[norm_pred]

    if any(kw in pred_clean for kw in ["развлеч", "кино", "музык", "entertain", "развлечения"]):
        return "entertainment"
    if any(kw in pred_clean for kw in ["географ", "гео", "geograph", "map"]):
        return "geography"
    if any(kw in pred_clean for kw in ["здоров", "медицина", "болезнь", "health", "medicine"]):
        return "health"
    if any(kw in pred_clean for kw in ["политик", "парти", "закон", "politic", "law"]):
        return "politics"
    if any(kw in pred_clean for kw in ["наук", "технолог", "tech", "science", "computer"]):
        return "science/technology"
    if any(kw in pred_clean for kw in ["спорт", "футбол", "игр", "sport", "game"]):
        return "sports"
    if any(kw in pred_clean for kw in ["путешеств", "туризм", "travel", "trip"]):
        return "travel"

    return "science/technology"  

y_pred_clean = [normalize_and_map(pred) for pred in y_pred_raw]

In [ ]:
PROMPT_VERSIONS = [1, 2, 3]


def majority_vote(labels: List[str]) -> str:
    counts = Counter(labels)
    return counts.most_common(1)[0][0]


ensemble_val_preds = []
for text in tqdm(validation_set['text'], desc="Ensemble prompts (val)"):
    per_prompt_labels = []
    for v in PROMPT_VERSIONS:
        msgs = build_messages_with_prompt_version(text, examples_by_categories, version=v)
        try:
            output = llm_pipeline(
                msgs,
                max_new_tokens=20,
                do_sample=True,
                temperature=0.2,
                top_p=0.7,
            )
            raw = output[0]['generated_text'].strip()
        except Exception as e:
            print(f"Error (val, v={v}): {e}")
            raw = ""
        label = parse_llm_output(raw, list_of_categories)
        per_prompt_labels.append(label)

    final_label = majority_vote(per_prompt_labels)
    ensemble_val_preds.append(final_label)

print("\nEnsemble по промптам - метрики на валидации:\n")
print(classification_report(y_true=validation_set['category'], y_pred=ensemble_val_preds))


ensemble_test_preds = []
for text in tqdm(test_set['text'], desc="Ensemble prompts (test)"):
    per_prompt_labels = []
    for v in PROMPT_VERSIONS:
        msgs = build_messages_with_prompt_version(text, examples_by_categories, version=v)
        try:
            output = llm_pipeline(
                msgs,
                max_new_tokens=20,
                do_sample=True,
                temperature=0.2,
                top_p=0.7,
            )
            raw = output[0]['generated_text'].strip()
        except Exception as e:
            print(f"Error (test, v={v}): {e}")
            raw = ""
        label = parse_llm_output(raw, list_of_categories)
        per_prompt_labels.append(label)

    final_label = majority_vote(per_prompt_labels)
    ensemble_test_preds.append(final_label)

print("\nEnsemble по промптам - метрики на тесте:\n")
print(classification_report(y_true=test_set['category'], y_pred=ensemble_test_preds))


Ensemble prompts (val): 100%|██████████| 99/99 [10:36<00:00,  6.43s/it]



Ensemble по промптам — метрики на валидации:

                    precision    recall  f1-score   support

     entertainment       0.28      0.56      0.37         9
         geography       0.26      0.88      0.40         8
            health       0.83      0.45      0.59        11
          politics       1.00      0.43      0.60        14
science/technology       0.62      0.40      0.49        25
            sports       0.89      0.67      0.76        12
            travel       0.53      0.45      0.49        20

          accuracy                           0.51        99
         macro avg       0.63      0.55      0.53        99
      weighted avg       0.65      0.51      0.53        99



Ensemble prompts (test): 100%|██████████| 204/204 [22:05<00:00,  6.50s/it]


Ensemble по промптам — метрики на тесте:

                    precision    recall  f1-score   support

     entertainment       0.26      0.47      0.34        19
         geography       0.32      0.88      0.47        17
            health       0.81      0.59      0.68        22
          politics       1.00      0.47      0.64        30
science/technology       0.86      0.47      0.61        51
            sports       0.90      0.72      0.80        25
            travel       0.69      0.78      0.73        40

          accuracy                           0.61       204
         macro avg       0.69      0.63      0.61       204
      weighted avg       0.75      0.61      0.63       204



In [28]:
print(classification_report(y_true=y_true, y_pred=y_pred_clean))

                    precision    recall  f1-score   support

     entertainment       0.75      0.33      0.46         9
         geography       0.37      0.88      0.52         8
            health       0.75      0.55      0.63        11
          politics       1.00      0.29      0.44        14
science/technology       0.50      0.72      0.59        25
            sports       0.57      0.67      0.62        12
            travel       0.57      0.40      0.47        20

          accuracy                           0.55        99
         macro avg       0.64      0.55      0.53        99
      weighted avg       0.63      0.55      0.54        99

